# Расчет цены на автомобиль
### Книга 06 - тестирование моделей

Постановка задачи:
Необходимо разработать модель, которая бы рассчитывала цену на выставленный на продажу автомобиль.  По условиям учебной задачи обучающих данных в явном виде не предоставлено. Только тестовые, собранные на авто-ру больше года назад. Необходимо самостоятельно разработать программу, которая бы собирала данные по объявлениям на том же сайте авто.ру. Дополнительная сложность - количество данных. Оцениваться работа будет по порядка 35к записей. Необходимо собрать порядка 140 тыс записей.  На самом сайте автору сейчас актуально порядка 90к объявлений.

Краткие итоги предыдущей работы:
- С сайта авто.ру загружен набор предложений по 12 брендам автомобилей (как в тестовом наборе), размером порядка 42к записей.
- Корректировка инфляции показала что эффективнее всего просто домножать результат текущего прогнозирования цен на мультипликатор. Текущее значение мультипликатора 0.72
- Признаки подгружены из внешнего файла и созданы на основе существующих.
- В ходе экспериментов целевое колидование высококардинальных признаков оказалось предпочтительным.
- Опробован ряд моделей ML.

Задача книги:

- Попробовать модели LGBMRegressor и XGBRegressor.




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.feature_selection import mutual_info_regression
# import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
# from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# import P05_01_lib as p05 # Модули с предыдущих рабочих листов в виде отдельной библиотеки
RANDOM_SEED = 42

Определение (вместо обычного импорта) нескольких функций

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

In [6]:
def eval_model(Xdf,ydf,model):
    was = time.perf_counter()
    X = Xdf.to_numpy()
    y = ydf.to_numpy()
    metric = 0
    N = 5
    # print('Eval:', end=' ')
    kf = KFold(n_splits=N, shuffle=True, random_state=RANDOM_SEED)
    for trn_index, tt_index in kf.split(X):
        X_trn = X[trn_index] ; X_tt =  X[tt_index]
        y_trn = y[trn_index] ; y_tt =  y[tt_index]
        model.fit(X_trn, np.log(y_trn))
        y_prd = np.exp(model.predict(X_tt))
        metric += mape(y_tt, y_prd) * 100.0
    metric /= N
    print('done. {:.4f} sec'.format(time.perf_counter() - was))
    return metric


### Загрузка данных


In [7]:
path = '/content/drive/MyDrive/ML05/'

In [14]:
df = pd.read_csv(path+'211203_Train_Test.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,power,mileage,doors,sell_id,num_owners,price,train,orig_license,left_steering,size_cat,sport_car,euro_car,premium,age,ann_mil,intensity,power_size,vol_power,fuel_B,fuel_D,fuel_E,fuel_G,fuel_H,transmission_AMT,transmission_AT,transmission_CVT,transmission_MT,gear_AWD,gear_FWD,gear_RWD,body_type_MPV,body_type_SUV,body_type_coupe,body_type_f_back,body_type_sedan,body_type_wagon,brand_tgt,color_tgt,model_tgt
0,105.0,74000.0,5,1100575026,3,0,0,1,1,3,0,1,0,6.0,12333.333333,0,35.000000,87.500000,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,-0.221593,0.230733,-0.060526
1,110.0,60563.0,5,1100549428,1,0,0,1,1,3,0,1,0,3.0,20187.666667,1,36.666667,68.750000,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,-0.221593,0.566096,-0.060526
2,152.0,88000.0,5,1100658222,1,0,0,1,1,4,0,1,0,6.0,14666.666667,0,38.000000,84.444444,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,-0.221593,0.325289,0.276302
3,110.0,95000.0,5,1100937408,1,0,0,1,1,3,0,1,0,6.0,15833.333333,1,36.666667,68.750000,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,-0.221593,0.367085,-0.060526
4,152.0,58536.0,5,1101037972,1,0,0,1,1,3,0,1,0,8.0,7317.000000,0,50.666667,84.444444,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,-0.221593,0.444131,-0.060526


In [16]:
# df.columns

In [17]:
dig_cat = ['euro_car', 'premium', 'orig_license','left_steering','num_owners', 'doors', 'intensity', 'sport_car', 'size_cat'  ]
num_cols = [ 'power', 'mileage', 'age', 'ann_mil',  'vol_power', 'power_size' ]
target = ['price']
low_card_cat = ['fuel_B', 'fuel_D', 'fuel_E', 'fuel_G', 'fuel_H', 'transmission_AMT', 'transmission_AT', 'transmission_CVT', 'transmission_MT', 'gear_AWD', 'gear_FWD', 'gear_RWD', 'body_type_MPV', 'body_type_SUV', 'body_type_coupe', 'body_type_f_back', 'body_type_sedan', 'body_type_wagon']
high_card_cat = ['brand_tgt', 'color_tgt', 'model_tgt']
model_cols = num_cols + low_card_cat + dig_cat + high_card_cat; print('Model Columns:', model_cols, '\n')

Model Columns: ['power', 'mileage', 'age', 'ann_mil', 'vol_power', 'power_size', 'fuel_B', 'fuel_D', 'fuel_E', 'fuel_G', 'fuel_H', 'transmission_AMT', 'transmission_AT', 'transmission_CVT', 'transmission_MT', 'gear_AWD', 'gear_FWD', 'gear_RWD', 'body_type_MPV', 'body_type_SUV', 'body_type_coupe', 'body_type_f_back', 'body_type_sedan', 'body_type_wagon', 'euro_car', 'premium', 'orig_license', 'left_steering', 'num_owners', 'doors', 'intensity', 'sport_car', 'size_cat', 'brand_tgt', 'color_tgt', 'model_tgt'] 



In [18]:
other_cols =  [x for x in list(df.columns) if x not in model_cols]
print('Other Columns ', other_cols)

Other Columns  ['sell_id', 'price', 'train']


### Регрессор LGBMRegressor

In [19]:
# cbr = CatBoostRegressor(iterations = 5000, random_seed = RANDOM_SEED, eval_metric='MAPE', custom_metric=['R2', 'MAE'],  silent=True, )
lgbmr = LGBMRegressor(random_state=RANDOM_SEED, max_depth=14, n_estimators=800)

eval_model(df.query('train==1')[model_cols], df.query('train==1').price, lgbmr)

done. 17.1899 sec


11.268484239761714

In [20]:
lgbmr.fit( df.query('train==1')[model_cols].to_numpy(), np.log(df.query('train==1').price.to_numpy()) )
subm = df.query('train==0')[['sell_id', 'price']]
subm['price'] = np.exp( lgbmr.predict( df.query('train==0')[model_cols].to_numpy() ))*0.72
subm.head()

,sell_id,price
0,1100575026,6.341390e+05
1,1100549428,9.283118e+05
2,1100658222,1.101823e+06
3,1100937408,7.882481e+05
4,1101037972,7.318952e+05


In [21]:
subm.to_csv(path+'211203_subm_LGBMR.csv', index=False)

Kaggle Score 15.13989

### Регрессор XGBRegressor

In [22]:
xgbr = XGBRegressor(random_state=RANDOM_SEED, max_depth=12, n_estimators=1000, learning_rate=0.03, objective='reg:squarederror', colsample_bytree=0.5, alpha=1)

eval_model(df.query('train==1')[model_cols], df.query('train==1').price, xgbr)

done. 457.0125 sec


10.866934348586643

In [23]:
xgbr.fit( df.query('train==1')[model_cols].to_numpy(), np.log(df.query('train==1').price.to_numpy()) )
subm = df.query('train==0')[['sell_id', 'price']]
subm['price'] = np.exp( xgbr.predict( df.query('train==0')[model_cols].to_numpy() ))*0.72
subm.head()

,sell_id,price
0,1100575026,665148.6875
1,1100549428,931233.2500
2,1100658222,991825.0625
3,1100937408,804405.6250
4,1101037972,712425.3750


In [24]:
subm.to_csv(path+'211203_subm_XGBR.csv', index=False)

Kaggle Score 14.83517

Выводы:  Модели конечно хорошие.  Но CatBoost от Яндекса работает лучше. 